In [43]:
import pandas as pd,seaborn as sns,matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import json
import requests
from bs4 import BeautifulSoup
from modules.functions import get_schedule,create_home_and_away_simple_dataframe,plot_game_trends
from modules.functions import calculate_possessions,get_agg_boxscore,get_game_timestamp_half,clean_dataframe,add_pie_to_dataframe
from IPython.display import clear_output
import time

ImportError: cannot import name 'add_pie_to_dataframe' from 'modules.functions' (c:\Users\FrankMathews\Desktop\Things\Repos\CBB_2023\modules\functions.py)

In [2]:
get_schedule("Purdue")

,DATE,OPPONENT,GAME_ID
2,"Tue, Nov 8",Milwaukee,401483405
3,"Fri, Nov 11",Austin Peay,401483406
4,"Tue, Nov 15",Marquette,401479696
5,"Thu, Nov 24",West Virginia,401483331
6,"Fri, Nov 25",Gonzaga,401486748
7,"Sun, Nov 27",Duke,401486751
8,"Wed, Nov 30",@ Florida State,401479679
9,"Sun, Dec 4",Minnesota,401484836
10,"Wed, Dec 7",Hofstra,401483407
11,"Sat, Dec 10",@ Nebraska,401484843


In [9]:
game_id = 401484924
simple_df_away,simple_df_home = create_home_and_away_simple_dataframe(game_id,disp = False)

In [ ]:
game_id = 401484924

n_tries = 0

try:
    test_df
except NameError as e:
    test_df = pd.DataFrame()

last_reported_time = None

while n_tries < 10:

    
    timestamp,half = get_game_timestamp_half(game_id)

    if timestamp != last_reported_time:
        last_reported_time = timestamp
        n_tries = 0
        clear_output()
        last_reported_time = timestamp
        temp_df = get_agg_boxscore(game_id = game_id,disp = True)
        temp_df = temp_df.transpose().rename_axis("Team").reset_index()
        temp_df['Half'] = half
        temp_df['Timestamp'] = timestamp
        test_df = pd.concat([test_df,temp_df])
        create_home_and_away_simple_dataframe(game_id,disp = True)
        time.sleep(np.random.randint(12,17))
    else:
        time.sleep(25)
        n_tries +=1

In [ ]:
plot_game_trends(test_df,half = half,color1='black',color2='green')